In [9]:
# ===Defined IPython widget: stopButton===
from IPython.display import display, Image
import IPython
import ipywidgets as widgets
import threading

stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

In [1]:
import time
import cv2 

# Display function
# ================
def view( button, source=0, rotate=0, process=(lambda x:x), fps=10 ):
    """
    Args:
    button: An IPywidget button to control the function. The display stops if button.value == True.
    Source: An optional integer or filename to specify the source of the video stream.
    Rotate: optional integer. Set to 0 by default (not rotated). Set to 1 will rotate by 90 deg.
    process: optional function that processes each frame through process(frame) before displaying.
        Set to identity function by default.
    """
    display_handle=display("Please wait...", display_id=True)
    
    cap = cv2.VideoCapture(source)
    cap.set(cv2.CAP_PROP_FPS, fps)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

    delay = 1 / fps
    
    while True:
        ret, frame = cap.read()
        if button.value==True or not ret:
            cap.release()
            display_handle.update(None)
            button.value = False
            break
        # frame = cv2.flip(frame, 1) # if your camera reverses your image

        # Rotate the frame if rotate==1
        if rotate:
            # frame = cv2.transpose(frame)
            frame = cv2.flip(frame, 0)
        
        frame = process(frame)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        time.sleep(delay)

In [2]:
def putText(frame, text, color = (0, 255, 0), position_x = 50, position_y = 50):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_position = (position_x, position_y)
    text_scale = 0.65
    text_color = color
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

In [ ]:
def cv2_resize_with_pad(image, target_height=None, target_width=None, pad_color=(0, 0, 0)):    
    original_height, original_width = image.shape[:2]

    if ((target_height is None) or (target_width is None)):
        target_height = max(original_height, original_width)
        target_width = target_height
    
    
    # Calculate the scaling factor and new dimensions while maintaining aspect ratio
    scale = min(target_width / original_width, target_height / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    # Create a new image with the target dimensions and the padding color
    padded_image = np.full((target_height, target_width, 3), pad_color, dtype=np.uint8)
    
    # Calculate the padding offsets
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2
    
    # Place the resized image in the center of the padded image
    padded_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image
    
    return padded_image